In [ ]:
!pip install tensorflow
!pip install scipy
!pip install pynrrd

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import pandas as pd
import SimpleITK as sitk
from matplotlib import pyplot as plt
from scipy import ndimage
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import math
from skimage.filters import threshold_mean
import cv2

In [ ]:
import SimpleITK as sitk
from ipywidgets import interact

print(sitk.Version())

In [ ]:
import os
dataset_path = "C:/Users/clarynse/HN_CHUS_Laryngeal_Cancer/Raw_images"

In [ ]:
# Check if required directories exist
print("Dataset Path Exists:", os.path.exists(dataset_path))

In [ ]:
# List files in Raw_images folder
print("\nFiles in Raw_images folder:", os.listdir(dataset_path))

In [ ]:
import SimpleITK as sitk
import os

# Define input and output paths
dicom_root = "C:/Users/clarynse/HN_CHUS_Laryngeal_Cancer/Raw_images"  # Root folder containing subfolders with DICOM images
output_dir = "C:/Users/clarynse/HN_CHUS_Laryngeal_Cancer/Processed_nii_images"  # Folder to save converted 3D images
os.makedirs(output_dir, exist_ok=True)

# Iterate through each patient folder
for patient_folder in os.listdir(dicom_root):
    dicom_path = os.path.join(dicom_root, patient_folder)
    
    if os.path.isdir(dicom_path):  # Ensure it's a directory
        reader = sitk.ImageSeriesReader()
        dicom_series = reader.GetGDCMSeriesFileNames(dicom_path)
        
        if not dicom_series:
            print(f"Skipping {patient_folder}, no DICOM files found.")
            continue

        reader.SetFileNames(dicom_series)
        image = reader.Execute()

        # Save as .nii
        nii_path = os.path.join(output_dir, f"{patient_folder}.nii.gz")
        sitk.WriteImage(image, nii_path)

        print(f"Saved {patient_folder} as .nii")

In [ ]:
#Extracting Metadata from the images

def extract_metadata(image_path):
    image = sitk.ReadImage(image_path)
    spacing = image.GetSpacing()  # (x, y, z) voxel spacing
    size = image.GetSize()  # (x, y, z) dimensions
    direction = image.GetDirection()  # Image orientation
    origin = image.GetOrigin()  # Physical coordinates of origin
    
    print(f"File: {image_path}")
    print(f"Voxel Spacing: {spacing}")
    print(f"Size: {size}")
    print(f"Direction: {direction}")
    print(f"Origin: {origin}")
    print("-" * 50)

# Apply to all converted images
for file in os.listdir(output_dir):
    if file.endswith(".nii.gz"):
        extract_metadata(os.path.join(output_dir, file))

In [ ]:
#Resampling the images to 1 mm slices

def resample_image(image, new_spacing=(1.0, 1.0, 1.0)):
    old_spacing = image.GetSpacing()
    old_size = image.GetSize()

    new_size = [
        int(round(old_size[i] * (old_spacing[i] / new_spacing[i])))
        for i in range(3)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    
    return resampler.Execute(image)

# Apply to all converted images
for file in os.listdir(output_dir):
    if file.endswith(".nii.gz"):
        img_path = os.path.join(output_dir, file)
        img = sitk.ReadImage(img_path)

        resampled_img = resample_image(img)
        resampled_path = img_path.replace(".", "_resampled.")
        sitk.WriteImage(resampled_img, resampled_path)
        print(f"Resampled {file} and saved to {resampled_path}")

In [ ]:
#Crop the Images
#We extract a subregion based on the given coordinates (x1=200, x2=320, y1=130, y2=250) and slice range (30-80).
def crop_image(image, x1=200, x2=320, y1=130, y2=250, z_start=30, z_end=80):
    array = sitk.GetArrayFromImage(image)  # Convert to numpy array
    cropped_array = array[z_start:z_end, y1:y2, x1:x2]  # Crop region
    
    cropped_image = sitk.GetImageFromArray(cropped_array)
    cropped_image.SetSpacing(image.GetSpacing())  # Preserve voxel spacing
    return cropped_image

# Apply cropping
for file in os.listdir(output_dir):
    if "resampled" in file and file.endswith(".nii"):
        img_path = os.path.join(output_dir, file)
        img = sitk.ReadImage(img_path)

        cropped_img = crop_image(img)
        cropped_path = img_path.replace("resampled", "cropped")
        sitk.WriteImage(cropped_img, cropped_path)
        print(f"Cropped {file} and saved to {cropped_path}")

In [ ]:
#Visualizing 3D Cropped Images
#Since a 3D image consists of multiple slices, we'll display a few representative slices using matplotlib.
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

def visualize_3d_image(image_path, num_slices=5):
    """
    Visualize multiple slices from a 3D cropped image.
    
    Parameters:
    - image_path: Path to the 3D NIfTI or NRRD image
    - num_slices: Number of slices to display across the volume
    """
    image = sitk.ReadImage(image_path)
    img_array = sitk.GetArrayFromImage(image)  # Convert to numpy array

    total_slices = img_array.shape[0]  # Number of slices in Z-direction
    slice_indices = np.linspace(0, total_slices - 1, num_slices, dtype=int)  # Select slices evenly spaced

    fig, axes = plt.subplots(1, num_slices, figsize=(15, 5))
    
    for i, slice_idx in enumerate(slice_indices):
        axes[i].imshow(img_array[slice_idx], cmap="gray")
        axes[i].set_title(f"Slice {slice_idx}")
        axes[i].axis("off")

    plt.tight_layout()
    plt.show()

# Example usage
cropped_image_path = "C:/Users/clarynse/HN_CHUS_Laryngeal_Cancer/Processed_nii_images/HN_CHUS_001_cropped.nii"  # Replace with your actual file
visualize_3d_image(cropped_image_path)

#This code selects num_slices (default: 5) slices from the 3D volume and displays them.